In [203]:
push!(LOAD_PATH,"../WebTools/src/")
using WebTools

using DataFrames,Dates,Distributions

In [204]:
elements=[l1*l2 for l1 in 'A':'Z' for l2 in 'A':'Z']

products="Product_".*[l1*l2 for l1 in 'A':'Z' for l2 in 'A':'Z']
clients="Client_".*[string(l1) for l1 in 'A':'Z']

product_price=Dict(p=>rand(0.1:0.1:100) for p in products)
client_disc=Dict(c=>rand([0.0,0.1,0.2,0.3,0.4]) for c in clients)
client_fraud=Dict(c=>rand()>0.95 ? true : false for c in clients);

In [211]:
nr=1000000
invoices=DataFrame()
invoices[:Date]=rand(Date(2019,1,1):Dates.Day(1):Date(2019,1,31),nr)
invoices[:Client]=rand(clients,nr)
invoices[:Product]=rand(products,nr)
invoices[:Qty]=rand(10:100,nr)
invoices[:UnitPriceGr]=map(x->product_price[x],invoices[:Product])
invoices[:Discount]=map(x->client_disc[x],invoices[:Client])
invoices[:UnitPriceReal]=invoices[:UnitPriceGr].*(1 .-invoices[:Discount])
invoices[:Noise]=[rand()>0.95 ? rand(LogNormal(0.01,0.025)) : 1.0 for r in 1:nr]
invoices[:Fraud]=map(x->client_fraud[x] ? rand(0.95:0.01:0.99) : 1.0,invoices[:Client])
invoices[:UnitPrice]=invoices[:UnitPriceReal].*invoices[:Noise].*invoices[:Fraud]
invoices[:Value]=invoices[:Qty].*invoices[:UnitPrice]

invoices;

In [213]:
summary=by(invoices,[:Client,:Product],df->DataFrame(Value=sum(df[:Value]),meanPrice=mean(df[:UnitPrice]),stdPrice=std(df[:UnitPrice])))
summary[:cvPrice]=summary[:stdPrice]./summary[:meanPrice]
summary[:Risk]=summary[:cvPrice].*summary[:Value];

In [221]:
opt=Dict("xAxis"=>Dict("type"=>"value"),"yAxis"=>Dict("type"=>"value"),
            "series"=>[])
push!(opt["series"],Dict("type"=>"scatter","data"=>map(x->Dict("value"=>[round(x[:cvPrice],digits=5),x[:Value],x[:Risk],x[:Client],x[:Product]]),eachrow(summary))))
opt["visualMap"]=Dict("dimension"=>2,"min"=>minimum(summary[:Risk]),"max"=>maximum(summary[:Risk]),"top"=>"middle","right"=>10,"calculable"=>true,"color"=>["#f01104","#2f8946"])
opt["title"]=Dict("text"=>"Risco Preço nas Vendas por Cliente/Produto","left"=>"center")
opt["yAxis"]["name"]="Volume Vendas"
opt["xAxis"]["name"]="Coef Var Preço"
ec=EChart(opt,width=1200,height=800);

In [222]:
open("sales_risk.html", "w") do f
    write(f,"""<!DOCTYPE html>
<html>
<head>
    <title>Demo</title>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <script src="https://cdnjs.cloudflare.com/ajax/libs/echarts/4.3.0/echarts-en.min.js"></script>
</head>""")
    write(f,"<body>")
    write(f,html(ec))
    write(f,"</body>")
    write(f,"""<script>myChart.setOption({"tooltip":{"padding":10,"borderColor":"#777","borderWidth":1,"backgroundColor":"#242","formatter":
                    function (params, ticket, callback) {
                        return "Cliente : "+params.value[3]+"<br>Produto : "+params.value[4]+"<br>Vendas : "+Intl.NumberFormat().format(params.value[1])+"<br>Coef Var Preço : "+params.value[0]
                        }
                    }},false,true);</script>
    """)                
    
end


433